# SageMaker built-in BlazingText(Supervised) example

1. [Introduction](#Introduction)  
2. [Development Environment and Data Preparation](#Development-Environment-and-Data-Preparation)
    1. [Setup](#Setup)  
    2. [Data Preparation](#Data-Preparation)
    3. [Preprocessing](#Preprocessing)
3. [Starting Sagemaker Training Job](#Starting-Sagemaker-Training-Job)  
    1. [Create BlazingText Container](#Create-BlazingText-Container)  
    2. [Creating an Estimator and start a training job](#Creating-an-Estimator-and-start-a-training-job)   
    3. [Training](#Training)
4. [Inference](#Inference)
    1. [(Optional) Batch Inference](#(Optional)-Batch-Inference)
    1. [Hosting / Inference](#Hosting-/-Inference)
5. [Stop / Close the Endpoint](#Stop-/-Close-the-Endpoint)

## Introduction

ビルトインアルゴリズムの一つであるBlazingTextでは、教師なしアルゴリズムであるWord2Vecと教師ありの分類アルゴリズムを提供しています。

ビルトインアルゴリズムを使用する場合、学習とデプロイに関連するコードのほとんどを開発者が意識する必要がなくなる点も利点となります。    

このノートブックでは、Amazon の商品レビューに対する感情分析、つまり、そのレビューが Positive (Rating が 5 or 4) か、Negative (Rating が 1 or 2)なのかを判定します。これは、文書を Positive か Negative に分類する2クラスの分類問題なので、**BlazingText**による教師あり学習を適用することができます。

## Data Preparation

Amazon の商品レビューデータセットは[Registry of Open Data on AWS](https://registry.opendata.aws/)で公開されており、 以下からダウンロード可能です。    
このノートブックでは、日本語のデータセットをダウンロードします。

- データセットの概要    
https://registry.opendata.aws/amazon-reviews/

- 日本語のデータセット(readme.htmlからたどることができます）    
https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz

以下では、データをダウンロードして解凍 (unzip) します。

In [ ]:
import urllib.request
import os
import gzip
import shutil

download_url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz" 
dir_name = "data"
file_name = "amazon_review.tsv.gz"
tsv_file_name = "amazon_review.tsv"
file_path = os.path.join(dir_name,file_name)
tsv_file_path = os.path.join(dir_name,tsv_file_name)

os.makedirs(dir_name, exist_ok=True)

if os.path.exists(file_path):
    print("File {} already exists. Skipped download.".format(file_name))
else:
    urllib.request.urlretrieve(download_url, file_path)
    print("File downloaded: {}".format(file_path))
    
if os.path.exists(tsv_file_path):
    print("File {} already exists. Skipped unzip.".format(tsv_file_name))
else:
    with gzip.open(file_path, mode='rb') as fin:
        with open(tsv_file_path, 'wb') as fout:
            shutil.copyfileobj(fin, fout)
            print("File uznipped: {}".format(tsv_file_path))

## Preprocessing

ダウンロードしたデータには学習に不要なデータや直接利用できないデータもあります。以下の前処理で利用できるようにします。

1. ダウンロードしたデータには不要なデータも含まれているので削除します。
2. 2クラス分類 (positive が 1, negative が 0)となるように評価データを加工し、レビューデータをMeCabを使ってスペース区切りのデータにします。
3. 学習データ、バリデーションデータ、テストデータに分けて、学習用にS3にデータをアップロードします。

### データの確認

タブ区切りの tsv ファイルを読んで1行目を表示してみます。

In [ ]:
import pandas as pd

df = pd.read_csv(tsv_file_path, sep ='\t')
df.head()

### 不要なデータの削除

今回利用しないデータは以下の2つです。必要なデータだけ選んで保存します。

- 評価データ `star_rating` と レビューのテキストデータ `review_body` 以外のデータ
- 評価が 3 のデータ (positive でも negative でもないデータ)

In [ ]:
df_pos_neg = df.loc[:, ["star_rating", "review_body"]]
df_pos_neg = df_pos_neg[df_pos_neg.star_rating != 3]

In [ ]:
df_pos_neg.head()

### MeCab のインストール

BlazingText は、文章をそのまま学習・推論に利用することはできず、語ごとにスペースで区切って利用する必要があります。これはスペースで区切られている英語などでは問題ありませんが、スペースで区切られていない日本語では追加の処理が必要になります。

ここでは、形態素とよばれる語の単位に分解（分かち書き）する形態素解析ツール MeCab を利用します。MeCab は pip でインストールして利用することができます。冒頭に`!`を入れることで、シェルコマンドを実行できます。`import MeCab` としても問題ないか確認しましょう。

In [ ]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install mecab-python3
!{sys.executable} -m pip install unidic-lite

import MeCab

### データの加工

BlazingText では以下のようなデータが必要です。

```
__label__1  私 は これ が 好き　です 。
__label__0  私 は これ が きらい　です 。
```

`__label__数字` は文書のラベルを表します。negative `__label__0`、positive なら `__label__1` とします。ラベル以降は、文書をスペース区切りにしたものですので、各文に対して MeCab による形態素解析を実行します。全文の処理に2, 3分必要になる場合があります。

In [ ]:
mecab = MeCab.Tagger("-Owakati")

def func_to_row(x):
    if x["star_rating"] < 3:
        label = '0'
    else:
        label = '1'
    x["star_rating"] = "__label__" + label
    x["review_body"] = mecab.parse(x["review_body"].replace('<br />', '')).replace('\n', '')
    return x

labeled_df = df_pos_neg.apply(lambda x: func_to_row(x), axis =1)

### データの分割

すべてのデータを学習データとすると、データを使って作成したモデルが良いのか悪いのか評価するデータが別途必要になります。
そこで、データを学習データ、バリデーションデータ、テストデータに分割して利用します。学習データはモデルの学習に利用し、バリデーションデータは学習時のモデルの評価に利用します。最終的に作成されたモデルに対してテストデータによる評価を行います。

`train_ratio` で設定した割合のデータを学習データとし、残ったデータをバリデーションとデータテストデータに分割して利用します。学習に利用する学習データとバリデーションデータは、後にSageMakerで利用するために、`savetxt` を利用してスペース区切りの csv に保存します。

In [ ]:
import numpy as np

data_size = len(labeled_df.index)
train_ratio = 0.8
train_index = np.random.choice(data_size, int(data_size*train_ratio), replace=False)
other_index = np.setdiff1d(np.arange(data_size), train_index)
valid_index = np.random.choice(other_index, int(len(other_index)/2), replace=False)
test_index = np.setdiff1d(np.arange(data_size), np.concatenate([train_index, valid_index]))

np.savetxt('train.csv',labeled_df.iloc[train_index].values, fmt="%s %s", delimiter=' ') 
np.savetxt('validation.csv',labeled_df.iloc[valid_index].values, fmt="%s %s", delimiter=' ') 

print("Data is splitted into:")
print("Training data: {} records.".format(len(train_index)))
print("Validation data: {} records.".format(len(valid_index)))
print("Test data: {} records.".format(len(test_index)))

### Uploading data to `sagemaker_session_bucket`

SageMaker での学習に利用するために、学習データとバリデーションデータを S3 にアップロードします。SageMaker Python SDK の upload_data を利用すると、S3 にファイルをアップロードできます。アップロード先のバケットは `sagemaker-{リージョン名}-{アカウントID}`で、バケットがない場合は自動作成されます。もし存在するバケットにアップロードする場合は、このバケット名を引数で指定できます。

アップロードが終われば、TrainingInput を利用して、アップロードしたファイルの content_type などを指定します。

In [ ]:
import sagemaker
from sagemaker.inputs import TrainingInput

sess = sagemaker.Session()

s3_train_data = sess.upload_data(path='train.csv', key_prefix='amazon-review-data')
s3_validation_data = sess.upload_data(path='validation.csv', key_prefix='amazon-review-data')
print("Training data is uploaded to {}".format(s3_train_data))
print("Validation data is uploaded to {}".format(s3_validation_data))

train_data = TrainingInput(s3_train_data, distribution='FullyReplicated', content_type='text/plain', s3_data_type='S3Prefix')
validation_data = TrainingInput(s3_validation_data, distribution='FullyReplicated', content_type='text/plain', s3_data_type='S3Prefix')
data_channels = {'train': train_data, 'validation': validation_data}

## Starting Sagemaker Training Job

BlazingText はビルトインアルゴリズムなので、アルゴリズムの実装は不要です。BlazingTextのコンテナイメージを呼び出して実行します。`get_image_uri` を利用すればコンテナイメージの URI を取得することができます。 取得した URI とそれを実行するインスタンスなどを指定して、Estimator を呼び出すことで学習の設定を行うことができます。

ビルトインアルゴリズムでは、実行内容を設定するいくつかのハイパーパラメータを設定する必要があります。BlazingText では `mode` のハイパーパラメータが必須です。テキスト分類を行う場合は `mode="supervised"` の指定が必要です。

BlazingTextは、FastTextテキスト分類器を拡張し、カスタムCUDAカーネルを使用してGPUアクセラレーションを活用しています。このモデルは、マルチコアCPUやGPUを使って数分で10億語以上の単語を学習することができ、最先端の深層学習テキスト分類アルゴリズムと同等のパフォーマンスを実現しています。詳細については、[algorithm documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext.html)または[the text classification notebook](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/blazingtext_text_classification_dbpedia/blazingtext_text_classification_dbpedia.ipynb)をご参照ください。

要約すると、BlazingTextでは以下のモードが、異なるタイプのインスタンスでサポートされています。


|          Modes         	| cbow (supports subwords training) 	| skipgram (supports subwords training) 	| batch_skipgram 	| supervised |
|:----------------------:	|:----:	|:--------:	|:--------------:	| :--------------:	|
|   Single CPU instance  	|   ✔  	|     ✔    	|        ✔       	|  ✔  |
|   Single GPU instance  	|   ✔  	|     ✔    	|         -       	|  ✔ (Instance with 1 GPU only)  |
| Multiple CPU instances 	|     - 	|        -  	|        ✔       	|  -   | |

最後に S3 のデータを指定して fit を呼べば学習を始めることができます。

## Create BlazingText Container

In [ ]:
import boto3

region_name = boto3.Session().region_name
container = sagemaker.image_uris.retrieve("blazingtext", region_name)
print('Using SageMaker BlazingText container: {} ({})'.format(container, region_name))

## Creating an Estimator and start a training job

In [ ]:
bt_model = sagemaker.estimator.Estimator(
    container,
    role=sagemaker.get_execution_role(),
    instance_count=1, 
    instance_type='ml.m5.xlarge',
    input_mode= 'File',
    sagemaker_session=sess
)

bt_model.set_hyperparameters(
    mode="supervised",
    epochs=10,
    vector_dim=10,
    early_stopping=True,
    patience=4,
    min_epochs=5
)

## Training

In [ ]:
bt_model.fit(inputs=data_channels, logs=True)

## Inference

## (Optional) Batch Inference
バッチ変換処理を使用してファイルに対して一括で推論を実行します。    
ここではすでにトークナイズ、単語の頻度表現へ変換済みの学習データを使用しますが、新規のデータへ適用する場合は別途実行する必要があります。

In [ ]:
'''
# ビルトインのBatch推論は1ファイル6MBまでの制限があるため、データを小さくします（実運用で大きなデータを扱う際はデータを分割する必要があります）
test_data = labeled_df.iloc[test_index][:1000]
print(test_data.shape)
test_data.head()
'''

データのフォーマットは学習時とは異なり、json形式 or jsonline形式となります

`content-type: application/json`
```json
{
 "instances": ["the movie was excellent", "i did not like the plot ."]
}
```
top-kを予測したい場合は以下のように変更します。
```json
{
 "instances": ["the movie was excellent", "i did not like the plot ."],
 "configuration": {"k": 2}
}
```

`content-type: application/jsonlines`
```jsonline
{"source": "source_0"}
{"source": "source_1"}
```
top-kを予測したい場合は以下のように変更します。
```jsonline
{"source": "source_0", "k": 2}
{"source": "source_1", "k": 3}
```

In [ ]:
'''
# json形式
import json

d = list(test_data.review_body)
with open('test.json', 'w') as f:
    for line in d:
        json.dump({"source": line}, f, ensure_ascii=False)
'''

In [ ]:
'''
# jsonline形式
import json

d = {"instances": list(test_data.review_body)}
with open('test.json', 'w') as f:
    json.dump(d, f, ensure_ascii=False)
'''

In [ ]:
'''
s3_test_data = sess.upload_data(path='test.json', key_prefix='amazon-review-data')
print("Test data is uploaded to {}".format(s3_test_data))
'''

In [ ]:
'''
bucket = sess.default_bucket()
output_path = f's3://{bucket}/amazon-review-data/output/blazingtext_batch_transform'
'''

In [ ]:
'''
transformer = bt_model.transformer(
    instance_count = 1, 
    instance_type = 'ml.m5.xlarge', 
    output_path = output_path,
    strategy = "MultiRecord",
)

transformer.transform(
    data = s3_test_data, 
    data_type = "S3Prefix", 
    content_type = "application/json", 
    split_type = "Line",
)
'''

### Download-predict-file-from-s3
バッチ推論されたアウトプットファイルをS3からダウンロードして、精度を確認します。

In [ ]:
'''
from sagemaker.s3 import S3Downloader, s3_path_join

# creating s3 uri for result file -> input file + .out
output_file = "test.json.out"
output_path = s3_path_join(output_path, output_file)

# download file
S3Downloader.download(output_path, '.')
'''

In [ ]:
'''
with open(output_file) as f:
    output = json.load(f)
    print(output)
'''

## Hosting / Inference

学習が終わると、作成されたモデルをデプロイして、推論を実行することができます。デプロイは deploy を呼び出すだけでできます。`---`といった出力があるときはデプロイ中で、`!`が出力されるとデプロイが完了です。

エンドポイントは json 形式でリクエストを受け付けますので、serializer の content_type に `application/json` を指定します。

In [ ]:
text_classifier = bt_model.deploy(initial_instance_count = 1,instance_type = 'ml.m5.xlarge')
text_classifier.serializer = sagemaker.serializers.IdentitySerializer(content_type = 'application/json')

デプロイが終わったら推論を実行してみましょう。ここでは negative なレビューを 5件、 positive なレビューを 5件ランダムに選択して推論を実行します。

In [ ]:
import json

num_test = 5
test_data = labeled_df.iloc[test_index]

neg_test_data = test_data[test_data.star_rating == '__label__0']
pos_test_data = test_data[test_data.star_rating == '__label__1']

neg_index = np.random.choice(neg_test_data.index, num_test)
pos_index = np.random.choice(pos_test_data.index, num_test)

neg_test_sentences = [text for text in neg_test_data.loc[neg_index]["review_body"].values]
payload = {"instances" : neg_test_sentences}
response = text_classifier.predict(json.dumps(payload))
predictions = json.loads(response)

for i, pred in enumerate(predictions):
    print("Ground Truth: {}, Prediction: {} (probability: {})"
                      .format(0, pred["label"][0][-1], pred["prob"]))
    print(neg_test_sentences[i].replace(' ', ''))
    print()
    
pos_test_sentences = [text for text in pos_test_data.loc[pos_index]["review_body"].values]
payload = {"instances" : pos_test_sentences}
response = text_classifier.predict(json.dumps(payload))
predictions = json.loads(response)

for i, pred in enumerate(predictions):
    print("Ground Truth: {}, Prediction: {} (probability: {})"
                      .format(1, pred["label"][0][-1], pred["prob"]))
    print(pos_test_sentences[i].replace(' ', ''))
    print()

## Stop / Close the Endpoint

In [ ]:
text_classifier.delete_endpoint()